# Exploration on MMT Retrieval repo

In [1]:
import os
HOME_DIR = "/home/ec2-user/SageMaker"
os.chdir(f"{HOME_DIR}/MMT-Retrieval/")

In [3]:
!pip install -r requirements.txt

  Using cached sentence_transformers-2.0.0-py3-none-any.whl
     |████████████████████████████████| 76 kB 6.9 MB/s  eta 0:00:01
  Using cached transformers-4.9.1-py3-none-any.whl (2.6 MB)
  Using cached torch-1.9.0-cp36-cp36m-manylinux1_x86_64.whl (831.4 MB)
     |████████████████████████████████| 22.1 MB 54.6 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 257 kB/s  eta 0:00:01
  Using cached sentencepiece-0.1.96-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached tokenizers-0.10.3-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [7]:
!pip install anytree

  Using cached anytree-2.8.0-py2.py3-none-any.whl (41 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [8]:
from mmt_retrieval.model.models import OSCAR, UNITER, M3P, ClassificationHead, Pooling
from mmt_retrieval import MultimodalTransformer

# Loading pre-trained Model

In [9]:
# Load pre-trained model M3P
path_model_m3p = os.path.join(HOME_DIR, "model_m3p/0_M3P")
pretrained_model = M3P(model_path = path_model_m3p)

# Run train test

See application notebook in MMT-Retrieval/examples/applications/Image_Search.ipynb

In [26]:
!pwd

/home/ec2-user/SageMaker/MMT-Retrieval


In [31]:
import yaml
import numpy as np
import random
from datetime import datetime

In [13]:
config = yaml.load(open("examples/experiments/super_config.yaml"), Loader=yaml.FullLoader)

In [27]:
# Manual modifications
config["model_path"] = "david-test"

In [21]:
# Seed
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])
random.seed(config["seed"])

In [22]:
# Model config
model_config = config["model"]

# Select model M3P for this demo
model_config["name"] = "m3p"

# Creating folders
model_folder_name = f"{model_config['name']}-{datetime.now().strftime('%Y-%m-%d_%H-%M')}"
model_save_path = os.path.join(config["model_path"], model_folder_name)
os.makedirs(model_save_path, exist_ok=True)

In [46]:
# <TODO: CHANGE HERE> Put the path of our own model
model_config["model_path"] = path_model_m3p
model_config["pretrained_model_path"] = path_model_m3p

### Train

In [47]:
# Indicate to Train
config.get("do_train", True)

'Train'

In [51]:
def build_model(model_config):
    if "legacy" not in model_config["name"]:
        if "oscar" in model_config["name"]:
            embedding_model = OSCAR(model_config["pretrained_model_path"],
                                    max_seq_length=model_config.get("max_seq_length", 70),
                                    max_image_seq_len=model_config.get("max_image_seq_len", 50))
        elif "m3p" in model_config["name"]:
            embedding_model = M3P(model_config["pretrained_model_path"],
                                  max_seq_length=model_config.get("max_seq_length", 128),
                                  max_image_seq_len=model_config.get("max_image_seq_len", 50))
        elif "villa" in model_config["name"] or "uniter" in model_config["name"]:
            embedding_model = UNITER(model_config["pretrained_model_path"],
                                     max_seq_length=model_config.get("max_seq_length", 70),
                                     max_image_seq_len=model_config.get("max_image_seq_len", 50))
        if model_config.get("half_layers", False):
            module_list = torch.nn.ModuleList()
            for i, layer in enumerate(embedding_model.auto_model.encoder.layer):
                if i % 2 == 0:
                    module_list.append(layer)
            embedding_model.auto_model.encoder.layer = module_list

        class_head = ClassificationHead(2, model_config.get("input_key", "pooled_cls_token_embeddings"), 768,
                                        model_config.get("classifier_type", "linear"),
                                        model_config.get("scaling_factor", 2))
        pooling_model = Pooling(768,
                                       pooling_mode_mean_tokens=model_config.get("mean", True),
                                       pooling_mode_cls_token=model_config.get("cls", False),
                                       pooling_mode_max_tokens=model_config.get("max", False))
        return MultimodalTransformer(modules=[embedding_model, class_head, pooling_model])

In [53]:
model_config

{'model_path': '/home/ec2-user/SageMaker/model_m3p/0_M3P',
 'model_path_cross': 'YOUR_CROSS_ENCODER',
 'model_path_embedding': 'YOUR_EMBEDDING_MODEL',
 'name': 'm3p',
 'pretrained_model_path': '/home/ec2-user/SageMaker/model_m3p/0_M3P',
 'max_seq_length': 70,
 'max_image_seq_len': 50,
 'input_key': 'pooled_cls_token_embeddings',
 'classifier_type': 'linear',
 'scaling_factor': 1}

In [54]:
embedding_model = M3P(
    model_config["pretrained_model_path"],
    max_seq_length=model_config.get("max_seq_length", 128),
    max_image_seq_len=model_config.get("max_image_seq_len", 50))

In [55]:
embedding_model

M3P(
  (auto_model): TransformerModel(
    (position_embeddings): Embedding(514, 768)
    (embeddings): Embedding(250002, 768, padding_idx=1)
    (layer_norm_emb): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (image_embeddings): BertImageEmbeddings(
      (image_embeddings): Linear(in_features=2048, out_features=768, bias=True)
      (image_distbution_embeddings): Linear(in_features=1600, out_features=768, bias=True)
      (image_location_embeddings): Linear(in_features=5, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (refine_embeddings): AoA_Refiner_Core(
      (layers): ModuleList(
        (0): AoA_Refiner_Layer(
          (self_attn): MultiHeadedDotAttention(
            (linears): ModuleList(
              (0): Linear(in_features=768, out_features=768, bias=True)
              (1): Linear(in_features=768, out_features=768, bias=True)
              (2): Linea

In [52]:
# Build model
model = build_model(model_config)

2021-08-06 19:18:40 - Use pytorch device: cuda
